In [1]:
import praw
import time
import pandas as pd
import csv
from praw.models import Submission
import os
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm 

In [2]:
# Define necessary names
SUBREDDIT_NAME = "FamilyIssues"       
MAX_POSTS = 10000                       
BATCH_SIZE = 100                       
CHECKPOINT_FILE = "checkpoint.txt"      
POSTS_CSV = "r_" + SUBREDDIT_NAME + "_data/" + "r_familyissue_posts.csv"                 
COMMENTS_CSV = "r_" + SUBREDDIT_NAME + "_data/" + "r_familyissue_comments.csv"           
REQUEST_DELAY = 1

In [3]:
# Load reddit api throgh praw
reddit = praw.Reddit(
    client_id="qg6qmpLE5NHZ98F71GBiiA",
    client_secret="5flQrSnLQjGwIoLQIrV_YFluxej9GQ",
    user_agent="script:my_script:v1.0 (by /u/NathanPalaiologos)"
)

In [4]:
# Set csv file

pd.DataFrame(columns=[
    "post_id", "title", "author", "content",
    "score", "num_comments", "created_utc", "url"
]).to_csv(POSTS_CSV, index=False)


pd.DataFrame(columns=[
    "post_id", "comment_id", "author", "body", 
    "score", "created_utc"
]).to_csv(COMMENTS_CSV, index=False)

In [5]:
posts_file = open("r_" + SUBREDDIT_NAME + "_data/" + "r_familyissue_posts.csv", "w", newline="", encoding="utf-8")
comments_file = open("r_" + SUBREDDIT_NAME + "_data/" + "r_familyissue_comments.csv", "w", newline="", encoding="utf-8")

In [6]:
posts_writer = csv.writer(posts_file)
comments_writer = csv.writer(comments_file)
posts_writer.writerow(["id", "title", "author", "selftext", "score", "num_comments", "created_utc", "url"])
comments_writer.writerow(["post_id", "comment_id", "author", "body", "score", "created_utc"])


50

In [7]:
subreddit = reddit.subreddit(SUBREDDIT_NAME)

In [8]:
# Process comments
def process_comment(comment, post_id, csv_writer):
    csv_writer.writerow([
        post_id,
        comment.id,
        comment.author,
        comment.body,
        comment.score,
        comment.created_utc
    ])
    for reply in comment.replies:
        process_comment(reply, post_id, csv_writer)

# collect data
try:
    for submission in subreddit.new(limit=10000):
        print(f"Posts: {submission.title}")
        posts_writer.writerow([
            submission.id,
            submission.title,
            submission.author,
            submission.selftext,
            submission.score,
            submission.num_comments,
            submission.created_utc,
            submission.url
        ])

        # comments collection
        submission.comments.replace_more(limit=10)
        for comment in submission.comments.list():
            process_comment(comment, submission.id, comments_writer)

        time.sleep(1)  # Set delay to avoid blocking
finally:
    posts_file.close()
    comments_file.close()

Posts: Family and politics
Posts: Dad laughing at rape joke
Posts: Adult children of emotionally immature parents.
Posts: Im scared to interact with my dad's side.
Posts: Brother given a house
Posts: Mum is always talking about health issues and asking me for money?
Posts: Should I put my husbands sister in her place?
Posts: Is there any excuse anymore?
Posts: please help me with my family.
Posts: My partner died and the first I hear from my brother is via my parents... he wants to use a tent I own
Posts: Family Issues between me and my dad
Posts: Should I out my creepy grandpa? If so, how?
Posts: Am I being unreasonable?
Posts: I hate my family
Posts: Forgive forget or walk away
Posts: Need advice…family trip turned into a costly mistake
Posts: TW - Vent what should I do I also want my mom to heal but she is dismissive and never wants to talk about it
Posts: Unwanted opinions
Posts: Family of Cheaters
Posts: "Out of sight, Out of mind"
Posts: My single mom feel lonely at home
Posts: W